In [11]:
import pandas as pd
from pyhive import hive

def execute_query(cursor, query, fetch=False):
    try:
        cursor.execute(query)
        if fetch:
            return cursor.fetchall()
    except Exception as e:
        print(e)
        return e

conn = hive.Connection(host="localhost", port=10000, username="hive")
cursor = conn.cursor()

execute_query(cursor, '''
SHOW DATABASES
''', True)

[('course',), ('default',)]

In [12]:
execute_query(cursor, '''
CREATE DATABASE IF NOT EXISTS course
''')

execute_query(cursor, '''
USE course
''')

execute_query(cursor, '''
CREATE TABLE IF NOT EXISTS my_table (
    col1 STRING,
    col2 ARRAY<STRING>,
    col3 STRING,
    col4 INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '.'
COLLECTION ITEMS TERMINATED BY ':'
LINES TERMINATED BY '\\n'
STORED AS TEXTFILE
''')

### Activity 1
1. Study the structure of data then use create statement to create a table with necessary number of columns.
2. Load the data into table.
3. View data in tables after loading.

In [13]:
execute_query(cursor, '''
CREATE TABLE IF NOT EXISTS orders (
    cust_id VARCHAR(20),
    cust_name VARCHAR(50),
    odr_date STRING,
    ship_date STRING,
    courier VARCHAR(20),
    recvd_date STRING,
    returned VARCHAR(5),
    returned_date STRING,
    return_reason VARCHAR(50)
) 
row format delimited fields terminated by ","
lines terminated by "\n" 
stored as textfile
''')

In [14]:
pd.DataFrame(execute_query(cursor, '''
SELECT * FROM orders
''', True))

""


In [15]:
execute_query(cursor, '''
LOAD DATA LOCAL INPATH '/opt/hive/assignment_create_table_2018.txt' INTO TABLE orders
''')

In [18]:
execute_query(cursor, '''
select concat(cust_id, '-', cust_name, '-', courier) from orders limit 5
''', True)

[('#cust_id-  #cust_name-#Courer',),
 ('GGYZ333519YS-Allison-Fedx',),
 ('GGYZ333519YS-Allison-Delhivery',),
 ('GGYZ333519YS-Allison-Fedx',),
 ('GGYZ333519YS-Allison-Ekart',)]